In [113]:

# coding: utf-8

# In[19]:

import keras
from keras.datasets import mnist
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten
from keras.layers import Conv2D, MaxPooling2D,Convolution2D,Activation
from keras import backend as K
import numpy as np
import cv2
import os
import os.path
import xlrd
from sklearn import svm
import numpy as np
import pdb
import csv
K.set_image_dim_ordering('tf')

def translate(image, x, y):
    # 定义平移矩阵
    M = np.float32([[1, 0, x], [0, 1, y]])
    shifted = cv2.warpAffine(image, M, (image.shape[1], image.shape[0]))
 
    return shifted



In [114]:
#!/usr/bin/python
# -*- coding: utf-8 -*-
from os import listdir
from os.path import isfile, isdir, join
from random import randrange
# 指定要列出所有檔案的目錄
mypath = "/Users/Jason/Documents/Scripts/RecognizeBingoNumberFromTV/Train/"

# 取得所有檔案與子目錄名稱
files = listdir(mypath)

labels = []
filelist = []
i = 0
# 以迴圈處理
for f in files:
  # 產生檔案的絕對路徑
  fullpath = join(mypath, f)
  # 判斷 fullpath 是檔案還是目錄
  if isfile(fullpath):
    i+=1
    labels.append(str(f).split("_")[0])
    filelist.append(str(f))
    # print("{index}. 檔案：{filename}".format(index = i,filename=f))
print (labels)

['5', '5', '2', '0', '0', '7', '9', '9', '0', '7', '8', '0', '2', '5', '2', '5', '5', '2', '2', '8', '7', '9', '9', '7', '8', '2', '2', '5', '2', '2', '5', '8', '6', '9', '.DS', '7', '7', '9', '6', '8', '5', '2', '2', '2', '5', '2', '2', '5', '8', '7', '0', '7', '4', '4', '7', '9', '7', '0', '8', '2', '5', '5', '2', '2', '1', '8', '9', '6', '6', '3', '4', '3', '3', '4', '3', '6', '6', '9', '8', '1', '8', '6', '1', '6', '6', '3', '4', '1', '1', '4', '3', '6', '6', '6', '1', '9', '8', '5', '8', '6', '6', '8', '9', '1', '8', '4', '4', '3', '4', '3', '4', '1', '8', '9', '8', '6', '6', '8', '5', '6', '6', '1', '8', '9', '7', '4', '3', '3', '4', '7', '9', '8', '1', '6', '6', '5', '5', '8', '6', '7', '3', '4', '1', '3', '1', '4', '3', '7', '6', '8', '5', '5', '5', '5', '1', '8', '1', '6', '9', '8', '7', '3', '4', '3', '3', '1', '4', '3', '7', '8', '9', '1', '6', '1', '5', '5', '5', '5', '6', '6', '1', '8', '7', '3', '4', '1', '3', '3', '1', '3', '3', '4', '7', '8', '6', '1', '6', '5', '5', '6

In [115]:

X = []
y = []
#讀取總共有幾張訓練圖
picnum = len(labels)
print ("picnum : ",picnum)
img = None

#縮放圖片大小
Max_Width=20
Max_Hight=20
kernel = np.ones((2,1), np.uint8)
for i in range(1,picnum):
    img = cv2.imread(mypath + "{filename}".format(filename = filelist[i]),0)

    try:
        img = cv2.resize(img, (Max_Width, Max_Hight), interpolation=cv2.INTER_CUBIC)
        img_dilate = img.copy()
        img_erode = img.copy()
        img_translate = img.copy()

        #隨機橫向或直向移動+-3個像素點
        img_translate = translate(img_translate,randrange(4),randrange(4))

        #腐蝕
        img_dilate = cv2.dilate(img_dilate, kernel, iterations = 1)

        #膨脹
        img_erode = cv2.erode(img_erode, kernel, iterations = 1)


        X.append(img_translate)    
        y.append(labels[i])

        X.append(img_dilate)    
        y.append(labels[i])

        X.append(img_erode)    
        y.append(labels[i])

        X.append(img)    
        y.append(labels[i])

    except:
        continue
print("Read all images and labels")
print (len(X))


picnum :  294
Read all images and labels
1168


In [116]:


#定義類別輸出神經元對應到的英文字母與數字
labeldict = {            '0':0,
						 '1':1,
						 '2':2,
						 '3':3,
						 '4':4,
						 '5':5,
						 '6':6,
						 '7':7,
						 '8':8,
						 '9':9}
num_classes = 10

X = np.array(X)


for i in range(len(y)):
    c = keras.utils.to_categorical(labeldict[y[i]], num_classes)   
    y[i] = c

print ("Done")

Done


In [117]:

y = np.array(y)




batch_size = 25




# img_rows, img_cols = 20, 80


x_train = X[:int(len(X)*0.8)]
y_train = y[:int(len(y)*0.8)]
x_test = X[int(len(X)*0.8):]
y_test = y[int(len(y)*0.8):]


#重新設定圖片的cols
# np.array(x_train).reshape(len(x_train), 1, img_rows, img_cols)/255




In [118]:

x_train = x_train.reshape(x_train.shape[0], Max_Width, Max_Hight, 1)
x_test = x_test.reshape(x_test.shape[0], Max_Width, Max_Hight, 1)

input_shape=(Max_Width, Max_Hight, 1)
x_train = 255 - x_train
x_test = 255 - x_test
x_train = x_train.astype('float32')
x_test = x_test.astype('float32')
x_train /= 255
x_test /= 255


#定義模型
model = Sequential() 
model.add(Convolution2D(32, (3, 3), input_shape=input_shape))
model.add(Activation('relu'))
model.add(Convolution2D(32, (3, 3)))  
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))   

model.add(Convolution2D(64, (3, 3))) 
model.add(Activation('relu'))     
model.add(Convolution2D(64, (3, 3)))     
model.add(Activation('relu'))     
model.add(MaxPooling2D(pool_size=(2, 2)))     
model.add(Dropout(0.25))
               
model.add(Flatten())
 
model.add(Dense(200)) 
model.add(Activation('relu')) 
model.add(Dropout(0.5))      
model.add(Dense(num_classes)) 
model.add(Activation('softmax'))

model.compile(loss=keras.losses.binary_crossentropy,
							optimizer=keras.optimizers.Adadelta(),
							metrics=['accuracy'])




In [119]:
#訓練次數
epochs = 10

#開始訓練
#將監督資料輸入後開始訓練
model.fit(x_train, y_train,
					batch_size=batch_size,
					epochs=epochs,
					verbose=1,
					validation_data=(x_test, y_test))




# 指定要列出所有檔案的目錄
ModelPath = "/Users/Jason/Documents/Scripts/RecognizeBingoNumberFromTV/Models/"

#儲存模型
model.save(ModelPath + '/model.h5')


#評估準確度
score = model.evaluate(x_test, y_test, verbose=0)
print('Test loss:', score[0])
print('Test accuracy:', score[1])



Train on 934 samples, validate on 234 samples
Epoch 1/10
934/934 [==============================] - 4s 5ms/step - loss: 0.3201 - acc: 0.9000 - val_loss: 0.3595 - val_acc: 0.9000
Epoch 2/10
934/934 [==============================] - 2s 2ms/step - loss: 0.2846 - acc: 0.8994 - val_loss: 0.2580 - val_acc: 0.9154
Epoch 3/10
934/934 [==============================] - 3s 3ms/step - loss: 0.2340 - acc: 0.9093 - val_loss: 0.1388 - val_acc: 0.9513
Epoch 4/10
934/934 [==============================] - 2s 3ms/step - loss: 0.1681 - acc: 0.9342 - val_loss: 0.1803 - val_acc: 0.9231
Epoch 5/10
934/934 [==============================] - 2s 3ms/step - loss: 0.1307 - acc: 0.9488 - val_loss: 0.1240 - val_acc: 0.9423
Epoch 6/10
934/934 [==============================] - 2s 3ms/step - loss: 0.0995 - acc: 0.9635 - val_loss: 0.0573 - val_acc: 0.9808
Epoch 7/10
934/934 [==============================] - 2s 2ms/step - loss: 0.0771 - acc: 0.9710 - val_loss: 0.0588 - val_acc: 0.9761
Epoch 8/10
934/934 [==========

In [120]:

# In[35]:

pred = model.predict(x_test,batch_size = 25)


# In[36]:

outdict = ['0','1','2','3','4','5','6','7','8','9']

Resultlables=[]
correct_num = 0
for i in range(pred.shape[0]):
	try:
		c0 = outdict[np.argmax(pred[i][:10])]
		captchaImg = c0
		#print c,lables[8000+i][1]
		Resultlables.append((str(len(picnum)+i),captchaImg))
		if captchaImg == lables[len(picnum)+i][1]:
			#每成功一次就在成功率的基數下+1，這樣才能估算成功率
			correct_num = correct_num + 1
	except:
		pass
#統計正確率 0-1.0
print ("Test Whole Accurate : ", float(correct_num)/len(pred))



Test Whole Accurate :  0.0
